In [26]:
from login import config

params = config()

In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import openai
# Insert openai API key to use GPT
openai.api_key = params['api_key']

# Creating a webdriver instance
driver = webdriver.Chrome()
# This instance will be used to log into LinkedIn

# Opening linkedIn's login page
driver.get("https://linkedin.com/uas/login")

# waiting for the page to load
time.sleep(5)

# entering username
username = driver.find_element(By.ID, "username")
# In case of an error, try changing the element tag used here.

# Enter Your Email Address
username.send_keys(params['user'])

# entering password
pword = driver.find_element(By.ID, "password")
# In case of an error, try changing the element tag used here.

# Enter Your Password
pword.send_keys(params['password'])

# Clicking on the log in button
# Format (syntax) of writing XPath --> //tagname[@attribute='value']
driver.find_element(By.XPATH, "//button[@type='submit']").click()
# In case of an error, try changing the XPath used here.

In [43]:
def search_job_in_page(job_list, cur_page_num):
    job_name = job_list.find_all('li', {'class':'ember-view'})
    system_msg="You are a career coach who understands job requirements"
    user_msg="List 5 key skills to be a great fit with this job description: "
    for row in job_name:
        try:
            job_title = row.find('a').contents[0].strip()
            company = row.find('div', {'class':'job-card-container__company-name'}).contents[0].strip()
            rowid = row.get('id')
            print('Clicking job card ...')
            driver.find_element(By.XPATH, "//li[@id='"+rowid+"']").click()
            time.sleep(5)
            src = driver.page_source
            soup = BeautifulSoup(src, 'html5lib')
            job_card = soup.find('div', {'class':'scaffold-layout__list-detail-inner'})
            print('Extracting level ...')
            level = job_card.find('li', {'class':'jobs-unified-top-card__job-insight'}).find('span').contents[2]
            print('Extracting location ...')
            location = job_card.find('span', {'class':'jobs-unified-top-card__bullet'}).contents[0].strip()
            print('Extracting time_posted ...')
            time_posted = job_card.find('span', {'class':'jobs-unified-top-card__posted-date'}).contents[0].strip()
            print('Extracting text ...')
            text = job_card.find('div', {'class':'jobs-description__content jobs-description-content'}).find('span').getText().strip()
            user_msg+=text
#             response = openai.ChatCompletion.create(
#                                                     model="gpt-3.5-turbo",
#                                                     messages=[{"role": "system", "content": system_msg},
#                                                               {"role": "user", "content": user_msg}]
#                                                    )
#             response = response["choices"][0]["message"]["content"]
            print('Job Title: ',job_title)
            print('Company: ', company)
            print('Location: ', location)
            print('Level: ',level)
            print('5 Key Skills Needed according to GPT: ')
            print(text)
#             print()
            job_dict['job_title'].append(job_title)
            job_dict['company'].append(company)
            job_dict['location'].append(location)
            job_dict['level'].append(level)
#             job_dict['skills'].append(response)
            job_dict['skills'].append(text)
        except: 
            try:
                p_num = int(row.find('button').attrs.get('aria-label')[-1:])
                if cur_page_num<p_num:
                    cur_page_num=p_num
                    break
                elif cur_page_num>=p_num:
                    continue
            except:
                continue
    return cur_page_num

In [44]:
def get_job_titles_company(last_page):
    for i in range(1, last_page+1):
        src = driver.page_source

        # Now using beautiful soup
        soup = BeautifulSoup(src, 'html5lib')

        job_list = soup.find('div', {'class':'scaffold-layout__list-detail-inner'})
        print('||Listing all job lists in page ', str(i)+'||')
        next_page_num = search_job_in_page(job_list,i)
        print()
        driver.find_element(By.XPATH, "//button[@aria-label='Page "+str(next_page_num)+"']").click()
        time.sleep(7)
    return next_page_num

In [45]:
recomend_url= 'https://www.linkedin.com/jobs/collections/recommended?lipi=urn%3Ali%3Apage%3Ad_flagship3_job_home%3BobHSV5i9RA6yU%2F%2F5gLL1pQ%3D%3D'
jobs_url = "https://www.linkedin.com/jobs/"
    
# driver.get(jobs_url)

driver.get(recomend_url)
time.sleep(3)
job_dict = {}
job_dict['job_title'] = []
job_dict['company'] = []
job_dict['location'] = []
job_dict['level'] = []
job_dict['skills'] = []
get_job_titles_company(1)

||Listing all job lists in page  1||
Clicking job card ...
Extracting level ...
Extracting location ...
Extracting time_posted ...
Extracting text ...
Job Title:  Junior Data Analyst  (@ Leading Advisory Firm)
Company:  Hatch
Location:  Sydney, New South Wales, Australia
Level:  Full-time
5 Key Skills Needed according to GPT: 
This is a Data Analyst role with Camphin Boston in Sydney. 🚀We offer our clients the highest level of advice, provided by business professionals with over 35 years of accounting, taxation and auditing experience in Sydney, Australia.-- Camphin Boston --Role Seniority - Junior · A role for someone with some basic skills, good motivation and ability to learn. Typically 1-2 years of experience is required.More about the Data Analyst role with Camphin Boston 👀🤩We are excited to offer an exceptional opportunity for an ambitious Data Analyst with 1 – 2 years’ experience to join our team.In this role you will:      Utilise skills in data management, analysis, modelling 

Extracting level ...
Extracting location ...
Extracting time_posted ...
Extracting text ...
Job Title:  Data Analyst (Enterprise Data Team)
Company:  GitLab
Location:  Australia
Level:  Full-time · Mid-Senior level
5 Key Skills Needed according to GPT: 
The GitLab DevSecOps platform empowers 100,000+ organizations to deliver software faster and more efficiently. We are one of the world’s largest all-remote companies with 2,000+ team members and values that foster a culture where people embrace the belief that everyone can contribute. Learn more about Life at GitLab .An overview of this roleThe Enterprise Data Team collaborates with every department at GitLab to develop trusted data products and responsibly harness the power of our data.As our new Data Analyst, you will examine and analyze complex data and will own the end-to-end process of developing trusted reports and data products - from business requirements gathering, to dashboard development, to the delivery of actionable insight

Extracting level ...
Extracting location ...
Extracting time_posted ...
Extracting text ...
Job Title:  Data Analyst
Company:  Interactiveinc
Location:  Sydney, New South Wales, Australia
Level:  <span class="white-space-pre"> </span>
5 Key Skills Needed according to GPT: 
The ideal candidate will use their passion for big data and analytics to provide insights to the business covering various topics. They will conduct both recurring and ad hoc analyses for business users.Responsibilities      Understand the day-to-day issues that the business faces, which can be better understood with dataCompile and analyze data related to business issuesGTM Tagging MigrationGA4 Property Setup and Event Tagging
Qualifications      Bachelor's or Master's degree in Statistics or Applied Mathematics or equivalent experience1 - 2 years' Data Analysis experienceProficient in HTML, CSS, SQL



2

In [204]:
job_dict

{'job_title': ['Junior Data Analyst',
  'Data Analyst',
  'Insight Analyst, Unloan, x15 Ventures',
  'Data and BI Operations Analyst',
  'BI/Reporting Analyst',
  'Junior Data Analyst',
  'Graduate Consultants - Data'],
 'company': ['Centurion',
  'Jobs via eFinancialCareers',
  'Commonwealth Bank',
  'Transport for NSW',
  'INTELLIGEN',
  'Sika',
  'Hydrogen Group'],
 'location': ['Perth, Western Australia, Australia',
  'Sydney, New South Wales, Australia',
  'Sydney, New South Wales, Australia',
  'Sydney, New South Wales, Australia',
  'Sydney, New South Wales, Australia',
  'Perth, Western Australia, Australia',
  'Sydney, New South Wales, Australia'],
 'level': ['Full-time · Mid-Senior level',
  'Full-time · Entry level',
  'Full-time · Mid-Senior level',
  'Full-time',
  'Contract · Associate',
  'Full-time · Mid-Senior level',
  'Full-time · Mid-Senior level'],
 'skills': ['IntroductionYour new roleWe have an exciting opportunity for a Junior Business Analyst to join our team, 

In [1]:
import pandas as pd

In [205]:
pd.DataFrame(job_dict)

,job_title,company,location,level,skills
0,Junior Data Analyst,Centurion,"Perth, Western Australia, Australia",Full-time · Mid-Senior level,IntroductionYour new roleWe have an exciting o...
1,Data Analyst,Jobs via eFinancialCareers,"Sydney, New South Wales, Australia",Full-time · Entry level,Join a reputable telco company on an initial 6...
2,"Insight Analyst, Unloan, x15 Ventures",Commonwealth Bank,"Sydney, New South Wales, Australia",Full-time · Mid-Senior level,"Who We AreWe’re Unloan, a new digital lending ..."
3,Data and BI Operations Analyst,Transport for NSW,"Sydney, New South Wales, Australia",Full-time,About UsTransport for NSW keeps our State movi...
4,BI/Reporting Analyst,INTELLIGEN,"Sydney, New South Wales, Australia",Contract · Associate,Great team and supportive culture Initial 6 mo...
5,Junior Data Analyst,Sika,"Perth, Western Australia, Australia",Full-time · Mid-Senior level,IntroductionYour new roleWe have an exciting o...
6,Graduate Consultants - Data,Hydrogen Group,"Sydney, New South Wales, Australia",Full-time · Mid-Senior level,Salary: $60-70k + super (training and certific...


In [2]:
df = pd.read_csv('result.txt')
df.head()

,job_title,company,location,level,skills
0,Junior Data Analyst (@ Leading Advisory Firm),Hatch,"Sydney, New South Wales, Australia",Full-time,1. Data Management and Analysis: The candidate...
1,Graduate - Data Analyst (@ Leading Advisory Firm),Hatch,"Sydney, New South Wales, Australia",Full-time,Based on the job description for the Data Anal...
2,Graduate - Data & Analytics,Ampol Australia,"Sydney, New South Wales, Australia",Full-time,"Based on the job descriptions provided, here a..."
3,Data Analyst (Enterprise Data Team),GitLab,Australia,Full-time · Mid-Senior level,Key skills for the Data Analyst role with Camp...
4,Data and AI Consultant,Brennan IT,"Sydney, New South Wales, Australia",Full-time · Mid-Senior level,Key skills for the Data Analyst role at Camphi...


In [25]:
print(df.iloc[2,4])

Based on the job descriptions provided, here are five key skills that would make a great fit for a Data Analyst role with Camphin Boston in Sydney:

1. Data Management and Analysis: Strong skills in data management, analysis, modeling, and visualization are essential for extracting actionable insights and effectively communicating them.

2. Technical Proficiency: Expertise in Power BI and SQL with experience using relational databases is desired. Experience in data warehousing and automated reporting using visualization platforms like Power BI is also beneficial.

3. Attention to Detail: The ability to identify and resolve technical issues, as well as accurately identify and rectify discrepancies or errors in information and deliverables, is crucial for success in this role.

4. Problem-solving: Skills in numerical problem-solving and the ability to identify and develop logical solutions to problems are important for analyzing complex data and offering concrete recommendations.

5. Col

# Experiment

In [34]:
src = driver.page_source

# Now using beautiful soup
soup = BeautifulSoup(src, 'html5lib')

job_list = soup.find('div', {'class':'scaffold-layout__list-detail-inner'})

text = job_list.find('div', {'class':'jobs-description__content jobs-description-content'}).find('span').getText().strip()

In [42]:
for row in job_list.find_all('li', {'class':'ember-view'}):
    print(row.get('id'))
    print(row.find('a').contents[0].strip())
    print('----------------------------This is new line----------------------------')

ember931
Junior Data Analyst  (@ Leading Advisory Firm)
----------------------------This is new line----------------------------
ember943
Data Analyst
----------------------------This is new line----------------------------
ember955
Data Analyst
----------------------------This is new line----------------------------
ember966
Data Analyst
----------------------------This is new line----------------------------
ember977
Data Analyst (Enterprise Data Team)
----------------------------This is new line----------------------------
ember988
Data Analyst
----------------------------This is new line----------------------------
ember999
Individual Life Graduate Program 2024
----------------------------This is new line----------------------------
ember1011
Customer Analytics Associate
----------------------------This is new line----------------------------
ember1012
2023 Summer Internship - Technology Interns (Sydney)
----------------------------This is new line----------------------------
ember

AttributeError: 'NoneType' object has no attribute 'contents'

In [ ]:
driver.find_element(By.XPATH, "//a[@aria-label='"+job_title+"']").click()

In [114]:
openai.api_key = params['api_key']

In [116]:
system_msg="You are a career coach who understands job requirements"
user_msg="List 5 key skills to be a great fit in this job description "+text
response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                        messages=[{"role": "system", "content": system_msg},
                                         {"role": "user", "content": user_msg}])

In [118]:
print(response["choices"][0]["message"]["content"])

1. Strong analytical skills: The role requires the ability to collect, compile, analyze, and interpret data. A great fit would have a strong analytical mindset and be able to extract insights from data to support decision-making.
2. Proficiency in data manipulation and visualization tools: The candidate should have excellent skills in Microsoft Excel and Microsoft Power BI. This includes the ability to manipulate data, perform statistical analysis, and create visual representations such as dashboards and reports.
3. Attention to detail and accuracy: Given the importance of data quality, it is crucial for the candidate to have a high level of attention to detail and the ability to maintain accuracy in data collection, analysis, and reporting.
4. Problem-solving and continuous improvement mindset: The role involves identifying potential improvements in reporting processes and automating tasks. Therefore, a great fit would be someone who has a proactive problem-solving mindset and is cons